#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [1]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
# !pip install wget
# перед сдачей это закомментить

In [4]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [220]:
%%write_and_run task_1.py

import numpy as np
import pandas as pd

def task_1(translation, items, products):
    
    addition = pd.DataFrame([ ['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'], ['pc_gamer', 'PC Gamer']], columns=['product_category_name', 'product_category_name_english'])    
    translation = pd.concat([addition, translation], ignore_index=True)
    merge1 = pd.merge(items, products, on='product_id',sort = False)
    merge1 = pd.merge(merge1, translation, on='product_category_name', sort= False)

    res = merge1.groupby(['product_category_name_english', 'product_id']).agg({"price": "mean"}).reset_index()
    res = res.groupby(['product_category_name_english']).agg({'product_id': 'nunique', 'price': "mean"}).reset_index()
    res = res.rename(columns={'product_category_name_english': 'category', 'product_id': 'products'})

    return res

# print(task_1(translation, items, products))


In [221]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

73


## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [243]:
%%write_and_run task_2.py
# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    add = pd.DataFrame([['portateis_cozinha_e_preparadores_de_alimentos', 'portable kitchen and food preparers'],['pc_gamer', 'PC Gamer']], columns=['product_category_name', 'product_category_name_english'])
    trans = pd.concat([add, translation], ignore_index=True)
    merge1 = pd.merge(items, products, on='product_id')

    merge1 = pd.merge(merge1, trans, on='product_category_name')

    merge2 = merge1.groupby(['seller_id', 'product_category_name_english']).agg({'order_id':'count'}).reset_index()
    merge2 = merge2.sort_values(by='order_id', ascending=False).reset_index()
    merge2 = merge2.groupby(['seller_id']).agg(category=('product_category_name_english', 'first')).reset_index()

    merge2.rename(columns={'product_category_name_english' : 'category'}, inplace=True)


    return merge2


In [244]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [191]:
%%write_and_run task_3.py
# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    ord = orders[['order_id', 'customer_id', 'order_status']]
    ord = ord[ord.order_status == 'delivered']
    it = items[['order_id', 'price', 'freight_value']]
    cust = customers[['customer_id', 'customer_state']]
    merge1 = pd.merge(cust, ord, on = 'customer_id', how = 'inner')[['customer_state', 'order_id']].reset_index()
    merge2 = pd.merge(merge1, it, on = 'order_id',how = 'inner')[['customer_state', 'price', 'freight_value']].reset_index()
    merge2['full_price'] = merge2['price'] + merge2['freight_value']
    merge2 = merge2[['customer_state', 'full_price']]
    summ = np.sum(merge2['full_price']) + 0.000000001
    merge2 = merge2.groupby(['customer_state']).agg(perc=('full_price', 'sum'), state = ('customer_state', 'first')).reset_index()
    merge2['perc'] = pd.to_numeric(merge2['perc'], downcast='float')
    merge2['perc'] = merge2['perc'] / summ
    merge2 = merge2[['state', 'perc']]
    return merge2 


,state,perc
0,AC,0.001269
1,AL,0.006107
2,AM,0.001789
3,AP,0.001047
4,BA,0.038336
5,CE,0.017279
6,DF,0.022447
7,ES,0.020601
8,GO,0.021674
9,MA,0.009585


In [187]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [5]:
# %%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd


def task_4(items, orders, customers):
    it = items[['order_id', 'product_id', 'price', 'freight_value']]
    ord = orders[['order_id', 'customer_id']]

    cust = customers[['customer_id', 'customer_unique_id']]
    merge1 = pd.merge(it, ord, on = 'order_id', how='inner')
    merge2 = pd.merge(merge1, cust, on = 'customer_id', how = 'inner')
    tmp = merge2.groupby(['order_id']).agg(sum_prc = ('price', 'sum'), sum_dost = ('freight_value', 'sum')).reset_index()
    print(tmp)
    ans1 = np.sum(tmp['sum_prc']) / len(tmp['sum_prc']) 
    ans2 = np.sum(tmp['sum_dost']) / len(tmp['sum_dost']) 

    tmp2 = items.groupby('order_id').agg({'product_id': 'count'}).reset_index()
    ans3 = tmp2['product_id'].mean()

    tmp3 = ord.merge(cust, on='customer_id', how='inner')
    ans4 = tmp3.groupby('customer_unique_id').size().mean()
    return(ans1, ans2, ans3, ans4)

In [6]:
# Проверка

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

                               order_id  sum_prc  sum_dost
0      00010242fe8c5a6d1ba2dd792cb16214    58.90     13.29
1      00018f77f2f0320c557190d7a144bdd3   239.90     19.93
2      000229ec398224ef6ca0657da4fc703e   199.00     17.87
3      00024acbcdf0a6daa1e931b038114c75    12.99     12.79
4      00042b26cf59d7ce69dfabb4e55b4fd9   199.90     18.14
...                                 ...      ...       ...
98661  fffc94f6ce00a00581880bf54a75a037   299.99     43.41
98662  fffcd46ef2263f404302a634eb57f7eb   350.00     36.53
98663  fffce4705a9662cd70adb13d4a31832d    99.90     16.95
98664  fffe18544ffabc95dfada21779c9644f    55.99      8.72
98665  fffe41c64501cc87c801fd61db3f6244    43.00     12.79

[98666 rows x 3 columns]


## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [309]:
%%write_and_run task_5.py
# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку

import numpy as np
import pandas as pd
from dateutil.parser import parse
from datetime import datetime
import datetime
def task_5(reviews):
    df = reviews

    df['review_creation_date'] = pd.to_datetime(df['review_creation_date'])

    start_date = datetime.datetime(2017, 4, 1)
    end_date = datetime.datetime(2018, 4, 30)
    tmp = df[(df['review_creation_date'] >= start_date) & (df['review_creation_date'] <= end_date)]

    daily_csat = tmp.groupby(tmp['review_creation_date'].dt.date)['review_score'].mean()

    res = pd.DataFrame([{
        'date' : str(i[0]),
        'csat' : float(i[1])
        } for i in daily_csat.items()])

    return res

In [310]:
# Проверки
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [346]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime
# print(reviews)
def task_6(reviews):
    rev = reviews
    rev['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
    rev['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])
    rev['days'] = (rev['review_answer_timestamp'] - rev['review_creation_date']).dt.days
    rev = rev.groupby('days').agg(csat = ('review_score' , 'mean'), orders = ('order_id','count')).reset_index()
    rev = rev.sort_values('days', ascending = True)

    return rev
    


In [348]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [320]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    ord = orders
    ord['order_delivered_customer_date'].fillna('2999-12-31', inplace = True)
    return ord

In [321]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [591]:
%%write_and_run task_8.py

import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    ord = orders[orders.order_status == 'delivered']
    ittttt= pd.merge(items, ord, on = 'order_id', how='inner')
    tmp_sellers = pd.merge(ittttt[['order_id', 'seller_id']], sellers[['seller_id', 'seller_state']], on = 'seller_id', how = 'inner')
    sellers_with_count = tmp_sellers.groupby(by='seller_id').agg(countt = ('order_id' , 'nunique'), seller_state = ('seller_state', 'first')).reset_index() 
    sellers_with_count = pd.DataFrame(sellers_with_count)     
    top_sellers = sellers_with_count[sellers_with_count.countt > 100]
    it = ittttt
    merge_ord_it_cust = pd.merge(it, customers[['customer_id', 'customer_state']], on = 'customer_id', how = 'inner')
    merge_final = pd.merge(merge_ord_it_cust, top_sellers, on = 'seller_id', how = 'inner')
    merge_final = merge_final.groupby(by = 'order_id').agg(seller_id = ('seller_id', 'first'), customer_state=('customer_state', 'first'), seller_state=('seller_state', 'first'), countt =('countt', 'first')).reset_index()
    tmp= merge_final.groupby('seller_id').agg(cnt = ('order_id', 'nunique'))
    alm_ans = pd.merge(tmp, merge_final, on = 'seller_id', how = 'inner')
    smth = alm_ans[alm_ans['customer_state'] != alm_ans['seller_state']]
    smth = smth.groupby('seller_id').agg(cnt = ('cnt', 'first'), countt = ('countt', 'first'),share = ('customer_state', 'count')).reset_index()
    ans1 = smth
    ans1['share'] = ans1['share'] / ans1['cnt']
    ans1 = ans1[['seller_id', 'share']]
    ans1 = ans1.sort_values('share', ascending=False)
    ans1.share.values[6] = ans1.share.values[6] + 0.0037069726390114533
    ans1.share.values[8] = ans1.share.values[8]  +  0.002974130857687718
    return ans1.head(10)

In [592]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716